In [8]:
# Importar as Bibliotecas

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import statsmodels.api as sm
import numpy as np

# Carregamento e Pré-Processamento de Dados

In [13]:
try:
    df = pd.read_csv('C:/Users/natal/nataliaguarnieri/proj_1_otimizacao_ecommerce_testeAB/data/ecommerce_events_data.csv')
    print('Dados carregados com sucesso!')
except FileNotFoundError:
    print("ERRO: Certifique-se de que o arquivo 'ecommerce_events_data.csv' está dentro da pasta 'data'.")
    exit()

Dados carregados com sucesso!


In [14]:
# Passo 1: Identificar usuários únicos e seus atributos

# Vamos usar o último evento de cada usuário para capturar seus atributos
user_attributes = df.sort_values('timestamp').drop_duplicates(subset='user_id', keep='last')
user_attributes = user_attributes[['user_id', 'group', 'device_type', 'traffic_source', 'location']].copy()

In [15]:
# Passo 2: Criar a variável binária de Sucesso (Comprou = 1, Não Comprou = 0)
users_who_purchased = df[df['event_type'] == 'purchase']['user_id'].unique()

user_attributes['converted'] = np.where(
    user_attributes['user_id'].isin(users_who_purchased),
    1,
    0
)

In [16]:
# Passo 3: Criar variáveis Dummy para o Modelo

# Renomeia o grupo 'test' para 1 e o 'control' para 0
user_attributes['is_test_group'] = np.where(user_attributes['group'] == 'test', 1, 0)

# Criar os dummies para os outros segmentos, removendo a primeira categoria para evitar multicolinearidade
df_model = pd.get_dummies(
    user_attributes,
    columns=['device_type', 'traffic_source', 'location'],
    drop_first=True,
    dtype=int
)

# Adicionar a constante (Intercept) necessária para o modelo statsmodels
df_model = sm.add_constant(df_model, prepend=False)

print("\nPré-processamento concluído.")
print("DataFrame pronto para modelagem (apenas usuários únicos, com variável 'converted' e dummies).")
print(f"Número total de usuários: {len(df_model)}")


Pré-processamento concluído.
DataFrame pronto para modelagem (apenas usuários únicos, com variável 'converted' e dummies).
Número total de usuários: 100000


# Criação do Modelo de Regressão Logística

In [17]:
# Variável Dependente (Y): O que queremos prever (conversão)
Y = df_model['converted']

# Variável Independente (X): O que vai prever a conversão
# Incluímos o grupo de teste e todas as variáveis de segmento (dummies)
# Removemos colunas que não são preditoras (user_id, group, etc.)
exclude_cols = ['user_id', 'group', 'converted', 'device_type', 'traffic_source', 'location']
X = df_model.drop(columns=exclude_cols, errors='ignore')

In [18]:
# Passo 1: Criação do Modelo

logit_model = sm.Logit(Y, X)

In [19]:
# Passo 2: Treinamento/Ajuste do Modelo

result = logit_model.fit()

print("\n" + "="*70)
print("RESUMO COMPLETO DA REGRESSÃO LOGÍSTICA (STATSMODELS)")
print("="*70)
print(result.summary())

Optimization terminated successfully.
         Current function value: 0.147923
         Iterations 8

RESUMO COMPLETO DA REGRESSÃO LOGÍSTICA (STATSMODELS)
                           Logit Regression Results                           
Dep. Variable:              converted   No. Observations:               100000
Model:                          Logit   Df Residuals:                    99988
Method:                           MLE   Df Model:                           11
Date:                Thu, 25 Sep 2025   Pseudo R-squ.:                0.006903
Time:                        11:15:24   Log-Likelihood:                -14792.
converged:                       True   LL-Null:                       -14895.
Covariance Type:            nonrobust   LLR p-value:                 4.991e-38
                              coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------
is_test_group               

# Interpretação com ODDS Ratios (Razão de Chances)

In [23]:
# Odds Ratios são mais fáceis de interpretar:
# - OR > 1.0: Aumenta a chance de conversão.
# - OR < 1.0: Diminui a chance de conversão.

print("\n" + "="*70)
print("ODDS RATIOS (RAZÃO DE CHANCES) E VALORES-P")
print("="*70)

# Cria um DataFrame com os resultados

odds_ratios = pd.DataFrame({
    'Odds Ratio (OR)': np.exp(result.params),
    'P-value': result.pvalues
})

# Formatação e Ordenação

odds_ratios = odds_ratios.sort_values(by='Odds Ratio (OR)', ascending=False)
odds_ratios = odds_ratios[odds_ratios['P-value'] < 0.05] # Filtra apenas o que é estatisticamente significativo

print("Variáveis que impactam significativamente a conversão (P-value < 0.05):")
print(odds_ratios.to_markdown(floatfmt=".4f"))


ODDS RATIOS (RAZÃO DE CHANCES) E VALORES-P
Variáveis que impactam significativamente a conversão (P-value < 0.05):
|               |   Odds Ratio (OR) |   P-value |
|:--------------|------------------:|----------:|
| is_test_group |            1.6515 |    0.0000 |
| const         |            0.0250 |    0.0000 |
